In [282]:
import pandas as pd
import numpy as np
import re
import unidecode
from unidecode import unidecode
import unicodedata
from functools import reduce
import ast
import gc

from sklearn.preprocessing import MultiLabelBinarizer

In [283]:
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows',None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_colwidth', None)
# Ajustar la configuración para mostrar más filas
#pd.set_option('display.max_rows', 500)

#inicializo la lista to replace
to_replace=['Null','Nan','nan','NULL',None]

In [284]:
dfbarrios= pd.read_excel('../02-datos-limpios/distr_barrios_codpostal.xlsx')


#dfcalles= pd.read_excel('../02-datos-limpios/CALLEJERO_VIGENTE_NUMERACIONES_202501.xlsx')

In [ ]:

dfi= pd.read_csv('./data/dfi1.csv',sep=";",index_col=False)

dfunion= pd.read_csv('./data/20250302_Union_Sil_Ale.csv',sep=";",index_col=False)

dfpm= pd.read_csv('./data/dfpisosmadrid_paraunir.csv',sep=";",index_col=False)
dfredpiso= pd.read_csv('./data/red_pisoparaunir.csv',index_col=False)

In [287]:
#14012024 juntar estos ficheros

dfi.to_csv("dfi1.csv",sep=";", index=False)
dfunion.to_csv("dfunionv1.csv",sep=";", index=False)
dfpm.to_csv("dfpmv1.csv", sep=";",index=False)
dfredpiso.to_csv("dfredpisov1.csv",sep=";", index=False)


In [288]:
#dfi=pd.read_csv("dfi1.csv",sep=";", index_col=False)

In [289]:

def obtener_valores_por_columna(df, columna,valor):
    """
    Devuelve los valores de las demás columnas de la fila donde la columna especificada tiene el valor dado.

    :param df: DataFrame de pandas
    :param columna: Nombre de la columna para buscar el valor
    :param valor: Valor único para buscar en la columna
    :return: Diccionario con los valores de las demás columnas
    """
    fila = df.loc[df[columna] == valor]
    if not fila.empty:
        return fila.iloc[0].to_dict()
    else:
        return {}

# Función para normalizar texto (eliminar acentos y convertir a minúsculas)
def normalizar_texto(texto):
 return unidecode(texto).lower()
 
def ponerMay(df,campo):
    df[campo]=dfredpiso[campo].str.strip().str.capitalize()

# Función para completar valores nulos en codigo_Postal
def completar_codigo_postal(df, dfbarrios):
    for i, row in df.iterrows():
        if pd.isnull(row['codigo_Postal']):  # Si el código postal es nulo
            # Buscar en dfbarrios el valor correspondiente
            cod_barrio = row['cod_barrio']
            codigo_postal_barrio = dfbarrios[dfbarrios['cod_barrio'] == cod_barrio]['codigo_Postal']
            if not codigo_postal_barrio.empty:
                df.at[i, 'codigo_Postal'] = codigo_postal_barrio.values[0]
    return df




In [290]:
dfredpiso.columns

Index(['titulo', 'precio', 'cantidad_visitas', 'fecha_publicacion',
       'referencia', 'm2_constr', 'm2_utiles', 'dormitorios', 'banos',
       'titularanuncio', 'descripcion_larga', 'amueblado', 'alarma',
       'num_pisos', 'num_plantas', 'puerta_seguridad', 'acceso_minusvalido',
       'porteria', 'terraza', 'zonas_verdes', 'planta', 'garaje', 'trastero',
       'gastos_comunidad', 'tiene_piscina', 'exterior', 'tipo_fachada',
       'tipo_suelo', 'calefaccion', 'aire_acondicionado', 'antiguedad',
       'ascensor', 'agua_caliente', 'estado', 'emisiones_CO2',
       'consumo_energia', 'armarios', 'orientacion', 'url', 'nombre_oficina',
       'nuda_propiedad', 'web', 'fecha_descarga', 'tipologia', 'calle',
       'distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'codigo_Postal'],
      dtype='object')

In [291]:
dfunion.columns

Index(['Unnamed: 0', 'title', 'url', 'precio_anterior', 'descuento', 'EUR/m2',
       'm2_constr', 'distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'zona',
       'calle', 'consumoce_ano', 'letra_ce', 'emisiones_co2', 'tipologia',
       'propiedad_completa', 'estado', 'amueblado', 'planta', 'dormitorios',
       'banos', 'balcon', 'terraza', 'garaje', 'trastero', 'ascensor',
       'porteria', 'calefaccion', 'antiguedad', 'plantas_edificio',
       'aire_acondicionado', 'jardin', 'inmueble_ingresos',
       'alquiler_opcion_a_compra', 'disponibilidad', 'nuda_propiedad',
       'sup_comercial', 'tipo_inmueble', 'tiene_armario', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/metal',
       'carpinteria_exterior_vidrio/pvc', 'carpinteria_exterior_vidrio/madera',
       'carpi

In [292]:
dfi.columns

Index(['title', 'url', 'P', 'precio', 'precio_anterior', 'descuento', 'EUR/m2',
       'm2_constr', 'ciudad', 'distrito', 'cod_distrito', 'barrio',
       'cod_barrio', 'zona', 'calle', 'title2', 'consumoce_ano', 'letrace',
       'emisiones_co2', 'tipologia', 'propiedad_completa', 'estado',
       'amueblado', 'planta', 'habitaciones', 'dormitorios', 'banos', 'balcon',
       'terraza', 'garaje', 'trastero', 'ascensor', 'porteria',
       'acceso_minusv', 'calefaccion', 'ano_construccion', 'plantas_edificio',
       'aire_acondicionado', 'jardin', 'gastos_comunidad', 'inmueble_ingresos',
       'alquiler_opcion_a_compra', 'disponibilidad', 'usufructo',
       'nuda_propiedad', 'derecho_superficie', 'sup_comercial',
       'tipo_inmueble', 'armario_empotrado', 'buhardilla', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera'

In [293]:
dfpm.columns

Index(['Unnamed: 0', 'url', 'title', 'precio', 'dormitorios', 'banos',
       'm2_constr', 'referencia', 'tipologia', 'planta', 'descripcion_larga',
       'area', 'piscina', 'garaje', 'jardin', 'fecha_publicacion',
       'num_visitas', 'calefaccion', 'trastero', 'amueblado', 'chimenea',
       'videoportero', 'alarma', 'aire_acondicionado', 'barrio', 'cod_barrio',
       'distrito', 'cod_distrito', 'codigo_Postal', 'nuda_propiedad',
       'tiene_piscina', 'letrace', 'estado', 'gastos_comunidad', 'ascensor',
       'ano_construccion', 'terraza', 'porteria', 'balcon', 'cocina',
       'inmueble_ingresos', 'portero', 'exterior', 'armarios',
       'plantas_edificio', 'lujo', 'orientacion_este', 'interior',
       'orietacion_oeste', 'orientacion_sur', 'aire_ acondicionado',
       'inmueble_ingresps', 'num_plantas', 'Lujo', 'calle', 'web',
       'fecha_descarga'],
      dtype='object')

In [294]:
dfredpiso.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 50 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   titulo              1460 non-null   object 
 1   precio              1460 non-null   int64  
 2   cantidad_visitas    1460 non-null   int64  
 3   fecha_publicacion   1460 non-null   object 
 4   referencia          1460 non-null   object 
 5   m2_constr           1460 non-null   int64  
 6   m2_utiles           1079 non-null   object 
 7   dormitorios         1458 non-null   float64
 8   banos               1456 non-null   float64
 9   titularanuncio      967 non-null    object 
 10  descripcion_larga   1454 non-null   object 
 11  amueblado           1 non-null      float64
 12  alarma              3 non-null      float64
 13  num_pisos           16 non-null     object 
 14  num_plantas         20 non-null     object 
 15  puerta_seguridad    18 non-null     float64
 16  acceso

In [295]:
dfpm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 873 entries, 0 to 872
Data columns (total 57 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           873 non-null    int64  
 1   url                  873 non-null    object 
 2   title                873 non-null    object 
 3   precio               873 non-null    int64  
 4   dormitorios          873 non-null    int64  
 5   banos                873 non-null    int64  
 6   m2_constr            873 non-null    float64
 7   referencia           873 non-null    int64  
 8   tipologia            873 non-null    object 
 9   planta               150 non-null    object 
 10  descripcion_larga    858 non-null    object 
 11  area                 747 non-null    object 
 12  piscina              858 non-null    float64
 13  garaje               858 non-null    float64
 14  jardin               858 non-null    float64
 15  fecha_publicacion    858 non-null    obj

In [296]:
dfunion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7872 entries, 0 to 7871
Data columns (total 77 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Unnamed: 0                                 3144 non-null   float64
 1   title                                      7872 non-null   object 
 2   url                                        7872 non-null   object 
 3   precio_anterior                            55 non-null     float64
 4   descuento                                  55 non-null     object 
 5   EUR/m2                                     7855 non-null   float64
 6   m2_constr                                  7865 non-null   float64
 7   distrito                                   7872 non-null   object 
 8   cod_distrito                               7872 non-null   int64  
 9   barrio                                     7853 non-null   object 
 10  cod_barrio              

In [297]:
dfi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 81 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   title                                      288 non-null    object 
 1   url                                        288 non-null    object 
 2   P                                          288 non-null    object 
 3   precio                                     288 non-null    int64  
 4   precio_anterior                            0 non-null      float64
 5   descuento                                  0 non-null      float64
 6   EUR/m2                                     288 non-null    float64
 7   m2_constr                                  288 non-null    int64  
 8   ciudad                                     288 non-null    object 
 9   distrito                                   288 non-null    object 
 10  cod_distrito              

In [298]:
dfpm["inmueble_ingresos"].value_counts(dropna=False)

inmueble_ingresos
NaN     870
1.00      3
Name: count, dtype: int64

In [299]:
dfpm["inmueble_ingresps"].value_counts(dropna=False)

inmueble_ingresps
NaN     872
1.00      1
Name: count, dtype: int64

In [300]:
# Reemplazar los nulos en 'inmueble_ingresos' con los valores de 'inmueble_ingresps'
dfpm['inmueble_ingresos'] = dfpm['inmueble_ingresos'].fillna(dfpm['inmueble_ingresps'])
dfpm=dfpm.drop(columns="inmueble_ingresps")

In [301]:
dfi["web"]="Indomio.com"
dfi["fecha_descarga"]="06/01/2024"

In [302]:
nuevos_nombres ={'armario_empotrado': 'tiene_armario','piscina':'tiene_piscina'
 }
dfi = dfi.rename(columns=nuevos_nombres)


dfi.rename(columns={'sistema_alarma':'alarma','read_all': 'descripcion_larga','acceso_minusv':'acceso_discapacitado','ano_construccion':'antiguedad','plantas_edificio':'num_plantas','cocina_1':'cocina'}, inplace=True)
celiminar=['title2','ciudad','P']
dfi.drop(columns=celiminar,inplace=True)
dfi['certificado_energetico'] = dfi['letrace'].apply(lambda x: 'Disponible' if pd.notna(x) and x != '' else 'No indicado')
# Reemplazar "Sí" por 1 y "No" por 0
dfi['trastero'] = dfi['trastero'].replace({'Sí': 1, 'No': 0})
dfi['terraza'] = dfi['terraza'].replace({'Sí': 1, 'No': 0})

celiminar=['Unnamed: 0']
dfpm.drop(columns=celiminar,inplace=True)
dfpm['num_plantas'] = dfpm['num_plantas'].fillna(dfpm['plantas_edificio'])

dfpm['aire_ acondicionado'] = dfpm['aire_ acondicionado'].astype(str)
dfpm['aire_acondicionado'] = dfpm['aire_acondicionado'].astype(str)
# Asignar el valor de 'aire_ acondicionado' a 'aire_acondicionado' cuando este sea nulo
dfpm.loc[dfpm['aire_acondicionado'].isnull() & dfpm['aire_ acondicionado'].notnull(), 'aire_acondicionado'] = dfpm['aire_ acondicionado']
dfpm.rename(columns={'orietacion_oeste': 'orientacion_oeste','lujo':'lujosa','ano_construccion':'antiguedad','area':'zona'},inplace=True)

celiminar=['plantas_edificio','aire_ acondicionado']
dfpm.drop(columns=celiminar,inplace=True)

dfredpiso.rename(columns={'titulo': 'title','cantidad_visitas':'num_visitas','emisiones_CO2':'emisiones_co2','acceso_minusvalido':'acceso_discapacitado'}, inplace=True)
dfunion.rename(columns={'plantas_edificio':'num_plantas'}, inplace=True)
celiminar=['Unnamed: 0']
dfunion.drop(columns=celiminar,inplace=True)

C:\Users\extas\AppData\Local\Temp\ipykernel_3828\2894162955.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfi['trastero'] = dfi['trastero'].replace({'Sí': 1, 'No': 0})
C:\Users\extas\AppData\Local\Temp\ipykernel_3828\2894162955.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfi['terraza'] = dfi['terraza'].replace({'Sí': 1, 'No': 0})


In [303]:
#creo la columna tiene armario
#dfpisos['tiene_armario'] = dfpisos['armarios'].apply(lambda x: 1 if pd.notna(x) and x != '' else np.nan)

ponerMay(dfredpiso,'barrio')
ponerMay(dfredpiso,'distrito')
ponerMay(dfunion,'barrio')
ponerMay(dfunion,'distrito')
ponerMay(dfi,'barrio')
ponerMay(dfi,'distrito')
ponerMay(dfpm,'barrio')
ponerMay(dfpm,'distrito')


In [304]:
duplicadas = dfpm.columns[dfpm.columns.duplicated()]
print("Columnas duplicadas:", duplicadas)


Columnas duplicadas: Index([], dtype='object')


In [305]:
dfpm.query("url=='https://www.pisosmadrid.com.es/propiedad/3890818-piso-en-venta-en-moratalaz-madrid'")

,url,title,precio,dormitorios,banos,m2_constr,referencia,tipologia,planta,descripcion_larga,zona,piscina,garaje,jardin,fecha_publicacion,num_visitas,calefaccion,trastero,amueblado,chimenea,videoportero,alarma,aire_acondicionado,barrio,cod_barrio,distrito,cod_distrito,codigo_Postal,nuda_propiedad,tiene_piscina,letrace,estado,gastos_comunidad,ascensor,antiguedad,terraza,porteria,balcon,cocina,inmueble_ingresos,portero,exterior,armarios,lujosa,orientacion_este,interior,orientacion_oeste,orientacion_sur,num_plantas,Lujo,calle,web,fecha_descarga
65,https://www.pisosmadrid.com.es/propiedad/3890818-piso-en-venta-en-moratalaz-madrid,"Piso en venta en Moratalaz, Madrid",198000,3,1,69.00,3890818,Piso,NaN,"Piso 3 dormitorios, 1 baños, 0 garajes, Para reformar, en Madrid, Madrid Cuarto piso sin ascensor, distribuido en salón-comedor, cocina, tres dormitorios y un baño.",Moratalaz,0.00,0.00,0.00,10/06/2024,11.00,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,Moratalaz,14.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pisosmadrid,24/10/2024


In [306]:
#dfpm.loc[dfpm['distrito'].isnull(), ['distrito', 'barrio']] = dfpm.loc[dfpm['distrito'].isnull()].apply(asignar_zona_distrito_barrio, axis=1)[['distrito', 'barrio']]


In [307]:
dfpm.query("cod_barrio.isnull()")[["url","barrio","distrito","tipologia"]]

,url,barrio,distrito,tipologia
24,https://www.pisosmadrid.com.es/propiedad/3872239-piso-en-venta-en-madrid,NaN,Centro,Piso
59,https://www.pisosmadrid.com.es/propiedad/3890776-piso-en-venta-en-vicalvaro-madrid,NaN,Vicálvaro,Piso
61,https://www.pisosmadrid.com.es/propiedad/3890778-piso-en-venta-en-madrid,NaN,NaN,Piso
65,https://www.pisosmadrid.com.es/propiedad/3890818-piso-en-venta-en-moratalaz-madrid,NaN,Moratalaz,Piso
84,https://www.pisosmadrid.com.es/propiedad/3895840-piso-en-venta-en-puente-de-vallecas-madrid,NaN,Puente De Vallecas,Piso
116,https://www.pisosmadrid.com.es/propiedad/3912810-piso-en-venta-en-salamanca-madrid,NaN,Salamanca,Piso
141,https://www.pisosmadrid.com.es/propiedad/3920110-piso-en-venta-en-tetuan-madrid,NaN,Tetuán,Piso
170,https://www.pisosmadrid.com.es/propiedad/3930160-piso-en-venta-en-puente-de-vallecas-madrid,NaN,Puente De Vallecas,Piso
175,https://www.pisosmadrid.com.es/propiedad/3931856-piso-en-venta-en-la-latina-madrid,NaN,Latina,Piso
214,https://www.pisosmadrid.com.es/propiedad/3948738-piso-en-venta-en-ciudad-lineal-madrid,NaN,Ciudad Lineal,Piso


dfunion=asignarbarriodistritodesdezona(dfunion)
dfredpiso=asignarbarriodistritodesdezona(dfredpiso)
dfi=asignarbarriodistritodesdezona(dfi)
dfpm=asignarbarriodistritodesdezona(dfpm)

In [308]:
dfredpiso['barrio'] = dfredpiso['barrio'].str.strip().str.capitalize()
dfredpiso['distrito'] = dfredpiso['distrito'].str.strip().str.capitalize()
dfunion['barrio'] = dfunion['barrio'].str.strip().str.capitalize()
dfunion['distrito'] = dfunion['distrito'].str.strip().str.capitalize()

dfpm.loc[dfpm['zona'] == 'Vallecas', 'distrito'] = 'Puente de Vallecas'
dfpm.loc[dfpm['zona'] == 'Moncloa', 'distrito'] = 'Moncloa-Aravaca'

dfpm.loc[dfpm['zona'] == 'Vallecas', 'distrito'] = 'Puente de Vallecas'
dfpm.loc[dfpm['zona'] == 'Moncloa', 'distrito'] = 'Moncloa-Aravaca'

In [309]:
dfbarrios["distrito"] = dfbarrios["distrito"].apply(lambda x: x.strip())


def procesar_distritos_nulos(df, dfbarrios):
    """
    Procesa las filas del DataFrame donde 'distrito' es nulo y devuelve 'cod_barrio', 'cod_distrito' y 'distrito'.
    
    :param df: DataFrame de pandas con los datos a procesar
    :param dfbarrios: DataFrame de referencia para obtener los valores
    :return: DataFrame con las columnas 'cod_barrio', 'cod_distrito' y 'distrito'
    """

    filas_nulas = df[df['distrito'].isna() & df['cod_barrio'].notna()]
    for idx, fila in filas_nulas.iterrows():
        valores = obtener_valores_por_columna(dfbarrios, 'nombre_barrio', fila['cod_barrio'])
        if valores:
             df.at[idx, 'barrio'] = valores.get('barrio')
             df.at[idx, 'cod_distrito'] = valores.get('cod_distrito')
             df.at[idx, 'distrito'] = valores.get('nombre_distrito')

In [310]:
# Asignar el valor a las filas de la 7765 a la 7771 en la columna 'zona'
#dfunion.loc[7765:7771, 'zona'] = "zona centro histórico vicalvaro"
#dfunion.loc[7765:7771, 'cod_barrio']=191

In [311]:
dfi.shape,dfpm.shape,dfunion.shape,dfredpiso.shape

((288, 81), (873, 53), (7872, 76), (1460, 50))

# comienza el merge

In [312]:
dfunion.shape,dfredpiso.shape,dfi.shape,dfpm.shape

((7872, 76), (1460, 50), (288, 81), (873, 53))

In [313]:

# Hacer un reset_index para cada DataFrame
dfunion = dfunion.reset_index(drop=True)
dfredpiso = dfredpiso.reset_index(drop=True)
dfi = dfi.reset_index(drop=True)
dfpm = dfpm.reset_index(drop=True)

# Concatenar los DataFrames
#df_unido = pd.concat([dfunion, dfredpiso, dfi, dfpm], ignore_index=True)
df_unido = pd.concat([dfunion, dfredpiso,dfpm], ignore_index=True)




In [314]:
df_unido.shape

(10205, 94)

In [315]:
df_unido['cod_barrio'] = df_unido['cod_barrio'].replace(0, np.nan)

In [316]:
#df_union = df_union.drop(columns=['disponibilidad'])
#df_union = df_union.drop(columns=['usufructo'])
#df_union = df_union.drop(columns=['derecho_superficie'])
#df_union = df_union.drop(columns=['buhardilla'])
#df_union = df_union.drop(columns=['hidromasaje'])

In [317]:
df_unido["jardin"].value_counts(dropna=False)

jardin
 NaN            8087
 Sin jardín      857
0.00             718
 Privado         245
 Comunitario     158
1.00             140
Name: count, dtype: int64

In [318]:

# Reemplazar los valores según las indicaciones
df_unido['piscina'] = df_unido['piscina'].replace({
    '-nan': 'NaN',
    'NaN': 'NaN',
    '-Comunitaria': 'Comunitaria',
    '-Propia': 'Privada',
    '-Con salorium': 'Comunitaria',
    '-1 piscina de adultos y 1 piscina de niños': 'Comunitaria'
})



In [319]:
df_unido['tiene_jardin'].value_counts(dropna=False)

tiene_jardin
NaN     8945
0.00     857
1.00     403
Name: count, dtype: int64

In [320]:
df_unido['jardin'].value_counts(dropna=False)

jardin
 NaN            8087
 Sin jardín      857
0.00             718
 Privado         245
 Comunitario     158
1.00             140
Name: count, dtype: int64

In [321]:
# Asegurarse de que todos los valores sean cadenas, manejar valores NaN
df_unido['jardin'] = df_unido['jardin'].astype(str).str.lower().apply(unidecode).str.strip().replace('nan', None)

In [322]:
df_unido['jardin']=df_unido['jardin'].replace('0.0','sin jardin')

In [323]:
df_unido['jardin'].value_counts(dropna=False)

jardin
None           8087
sin jardin     1575
privado         245
comunitario     158
1.0             140
Name: count, dtype: int64

In [324]:
# Aplicar la lógica solo a los valores nulos en 'tiene_jardin'
df_unido['tiene_jardin'] = df_unido['tiene_jardin'].where(
    pd.notnull(df_unido['tiene_jardin']),  # Mantener valores no nulos existentes
    df_unido['jardin'].apply(lambda x: 1 if pd.notnull(x) and x != 'sin jardin' else (0 if x == 'sin jardin' else None))
)

# Verificar los resultados
print(df_unido[['jardin', 'tiene_jardin']].head())


       jardin  tiene_jardin
0        None           NaN
1        None           NaN
2     privado          1.00
3  sin jardin          0.00
4  sin jardin          0.00


In [325]:
df_unido['tiene_jardin'].value_counts(dropna=False)

tiene_jardin
NaN     8087
0.00    1575
1.00     543
Name: count, dtype: int64

In [326]:
df_unido['jardin'].value_counts(dropna=False)

jardin
None           8087
sin jardin     1575
privado         245
comunitario     158
1.0             140
Name: count, dtype: int64

In [327]:
df_unido['jardin']=df_unido['jardin'].replace('1.0','comunitario')

In [328]:
df_unido['jardin'].value_counts(dropna=False)

jardin
None           8087
sin jardin     1575
comunitario     298
privado         245
Name: count, dtype: int64

In [329]:
df_unido["tipologia"].value_counts(dropna=False)

tipologia
Piso           8673
Ático           368
Chalet          294
Casa            266
Apartamento     219
Dúplex          160
Estudio         111
Atico            45
Loft             31
Duplex           15
Adosado          12
Negocio           4
Pareado           3
Venta             2
Villa             1
Bungalow          1
Name: count, dtype: int64

In [330]:
#Reemplazar todas las tipologías que comienzan con "Casa" por "Casa"
df_unido['tipologia'] = df_unido['tipologia'].replace(r'^Casa.*', 'Casa', regex=True)
df_unido['tipologia'] = df_unido['tipologia'].replace(r'^Apartamento.*', 'Apartamento', regex=True)
df_unido['tipologia'] = df_unido['tipologia'].replace(r'^Caserío.*', 'Casa', regex=True)

In [331]:

# Eliminar las filas donde 'precio' es nulo
df_unido = df_unido.dropna(subset=['precio'])

In [332]:
df_unido.columns

Index(['title', 'url', 'precio_anterior', 'descuento', 'EUR/m2', 'm2_constr',
       'distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'zona', 'calle',
       'consumoce_ano', 'letra_ce', 'emisiones_co2', 'tipologia',
       'propiedad_completa', 'estado', 'amueblado', 'planta', 'dormitorios',
       'banos', 'balcon', 'terraza', 'garaje', 'trastero', 'ascensor',
       'porteria', 'calefaccion', 'antiguedad', 'num_plantas',
       'aire_acondicionado', 'jardin', 'inmueble_ingresos',
       'alquiler_opcion_a_compra', 'disponibilidad', 'nuda_propiedad',
       'sup_comercial', 'tipo_inmueble', 'tiene_armario', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/metal',
       'carpinteria_exterior_vidrio/pvc', 'carpinteria_exterior_vidrio/madera',
       'carpinteria_exterior_vid

In [333]:
# Forzar la recolección de basura
gc.collect()


1886

In [334]:
df_unido.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10205 entries, 0 to 10204
Data columns (total 94 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   title                                      10205 non-null  object 
 1   url                                        10205 non-null  object 
 2   precio_anterior                            55 non-null     float64
 3   descuento                                  55 non-null     object 
 4   EUR/m2                                     7855 non-null   float64
 5   m2_constr                                  10198 non-null  float64
 6   distrito                                   10198 non-null  object 
 7   cod_distrito                               10198 non-null  float64
 8   barrio                                     10137 non-null  object 
 9   cod_barrio                                 10128 non-null  float64
 10  zona                  

In [335]:
#teniendo barrio, poner cod_distrito,distrito,codigoPostal

In [336]:
df_unido.query("url=='https://www.pisosmadrid.com.es/propiedad/3890818-piso-en-venta-en-moratalaz-madrid'")

,title,url,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,consumoce_ano,letra_ce,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,antiguedad,num_plantas,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,exterior,interior,interior_y_exterior,piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,€_comunidad_mes,acceso_discapacitado,orientacion,precio,web,fecha_descarga,actualizacion,armarios,codigo_Postal,m2_utiles,referencia,tipo_suelo,certificado_energetico,tiene_jardin,descripcion_larga,num_visitas,fecha_publicacion,titularanuncio,num_pisos,puerta_seguridad,zonas_verdes,gastos_comunidad,tiene_piscina,tipo_fachada,agua_caliente,consumo_energia,nombre_oficina,letrace,portero,orientacion_este,orientacion_oeste,orientacion_sur,Lujo
9397,"Piso en venta en Moratalaz, Madrid",https://www.pisosmadrid.com.es/propiedad/3890818-piso-en-venta-en-moratalaz-madrid,NaN,NaN,NaN,69.00,Moratalaz,14.00,NaN,NaN,Moratalaz,NaN,NaN,NaN,NaN,Piso,NaN,NaN,NaN,NaN,3.00,1,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,nan,sin jardin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,198000,pisosmadrid,24/10/2024,NaN,NaN,NaN,NaN,3890818,NaN,NaN,0.00,"Piso 3 dormitorios, 1 baños, 0 garajes, Para reformar, en Madrid, Madrid Cuarto piso sin ascensor, distribuido en salón-comedor, cocina, tres dormitorios y un baño.",11.00,10/06/2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [337]:
def normalize_string(s):
    if pd.notna(s):
        s = unicodedata.normalize('NFD', s)  # Descomponer los caracteres con acento
        s = ''.join([c for c in s if not unicodedata.combining(c)])  # Eliminar los caracteres de acento
        s = s.lower().strip().replace(" ", "")  # Convertir a minúsculas
    return s

In [338]:
# Cambiar el tipo de dato de las columnas "barrio" y "distrito" a object
df_unido["barrio"] = df_unido["barrio"].astype(object)
df_unido["distrito"] = df_unido["distrito"].astype(object)
df_unido["cod_barrio"] = df_unido["cod_barrio"].astype(object)
df_unido["cod_distrito"] = df_unido["cod_distrito"].astype(object)
df_unido["codigo_Postal"] = df_unido["codigo_Postal"].astype(object)
dfbarrios.loc[:, 'distrito_n'] = [normalize_string(valor.strip().replace(" ", "")) for valor in dfbarrios.loc[:, 'distrito']]
dfbarrios.loc[:, 'barrio_n'] = [normalize_string(valor.strip().replace(" ", "")) for valor in dfbarrios.loc[:, 'barrio']]

In [339]:
#nuevas_columnas= ["barrio","cod_barrio","distrito","cod_distrito","codigo_Postal"]
#df_pisos[nuevas_columnas]=np.nan


zona_barrio_dict = {
    'Sanchinarro': {     
        'barrio': 'Valdefuentes'
    },
    'San Andrés': {       
        'barrio': 'Villaverde Alto'
    },
    'Virgen del Cortijo': {      
        'barrio': 'Valdefuentes'
    },
    'Montecarmelo': {       
        'barrio': 'Mirasierra'
    }
    ,
    'Villaverde Bajo': {       
        'barrio': 'Los Rosales'
    },
    'villaverdebajo': {       
        'barrio': 'Los Rosales'
    }
}




def clasificar_distrito_y_barrio_por_columnas(df, df_barrios,campo):
    # Recorrer las filas del dataframe
    for idx, fila in df.iterrows():
        # Recorrer las columnas en orden de prioridad: titulo, nombre_oficina
        for columna in [campo]:
            valor = fila[columna]
            
            # Verificamos que el valor no sea nulo
            if pd.notna(valor):
                valor_normalizado = normalize_string(valor)

                 # Reemplazar valores específicos en la ubicación normalizada
                if 'sanandres' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('sanandres', 'villaverdealto')
                elif 'salvador' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('salvador', 'elsalvador')
                elif 'san andres' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('san andres', 'villaverdealto')
                elif 'palos de moguer' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('palos de moguer', 'palosdemoguer')
                elif 'virgendelcortijo-manoteras' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('virgendelcortijo-manoteras', 'valdefuentes')
                elif 'sanchinarro' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('sanchinarro', 'valdefuentes')
                elif 'montecarmelo-elgoloso' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('montecarmelo-elgoloso', 'elgoloso')
                elif 'montecarmelo' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('montecarmelo', 'mirasierra')
                elif 'ambroz' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('ambroz', 'cascohistoricodevicalvaro')
                elif 'carmenes' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('carmenes', 'loscarmenes')
                elif 'paudecarabanchel' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('paudecarabanchel', 'buenavista')
                elif 'malasaña' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('malasaña', 'universidad')
                #elif 'centrohistorico' in valor_normalizado:
                    #valor_normalizado = valor_normalizado.replace('centrohistorico', 'cascohistoricodevicalvaro')
                elif 'lastablas' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('lastablas', 'valverde')
                elif 'malasana' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('malasana', 'universidad')
                elif 'lavapies' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('lavapies', 'embajadores')
                elif 'bernabeu' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('bernabeu', 'elviso')
                elif 'villaverdebajo' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('villaverdebajo', 'losrosales')
                elif 'orense' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('orense', 'cuatrocaminos')
                elif 'aguilas' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('aguilas', 'lasaguilas')
                elif 'lasmercedes' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('lasmercedes', 'rejas')
                elif 'granvia' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('granvia', 'justicia')
                elif 'chueca' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('chueca', 'justicia')
                elif 'puertadehierro' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('puertadehierro', 'ciudaduniversitaria')
                
                # Verificamos si el valor normalizado contiene el nombre de un barrio
                barrio_encontrado = next((barrio for barrio in df_barrios["barrio_n"] if barrio in valor_normalizado), None)
                
                if barrio_encontrado and pd.isna(fila["barrio"]):
                    df.at[idx, "barrio"] = df_barrios.query(f"barrio_n == '{barrio_encontrado}'")["barrio"].values[0]  # Asignar el nombre del barrio a la columna "barrio"
                    # Buscar los valores cod_barrio, distrito y cod_distrito en df_barrios
                    result = df_barrios.query(f"barrio_n == '{barrio_encontrado}'")
                    if not result.empty:
                        cod_distrito = result["cod_distrito"].values[0]
                        cod_barrio = result["cod_barrio"].values[0]
                        codigo_Postal=result["codigo_Postal"].values[0]
                        distrito_result = result["distrito"].values[0]
                        df.at[idx, "cod_barrio"] = cod_barrio
                        df.at[idx, "cod_distrito"] = cod_distrito
                        df.at[idx, "distrito"] = distrito_result
                        df.at[idx, "codigo_Postal"] = codigo_Postal
                # Verificamos si el valor normalizado contiene el nombre de un distrito
                distrito_encontrado = next((distrito for distrito in df_barrios["distrito_n"] if distrito in valor_normalizado), None)
                if distrito_encontrado and pd.isna(fila["distrito"]):
                    df.at[idx, "distrito"] = df_barrios.query(f"distrito_n == '{distrito_encontrado}'")["distrito"].values[0]
                    result = df_barrios.query(f"distrito_n == '{distrito_encontrado}'")                    
                    cod_distrito = result["cod_distrito"].values[0]
                    df.at[idx, "cod_distrito"] = cod_distrito
            else:
                #zona_encontrado = next((barrio for barrio in df_barrios["barrio_n"] if barrio in valor_normalizado), None)
                barrio_encontrado = next((zona_barrio_dict[zona]["barrio"] for zona in zona_barrio_dict if zona_barrio_dict[zona]["barrio"] in df_barrios["barrio_n"]), None)
                if barrio_encontrado and pd.isna(fila["barrio"]):
                    df.at[idx, "barrio"] = df_barrios.query(f"barrio_n == '{barrio_encontrado}'")["barrio"].values[0]  # Asignar el nombre del barrio a la columna "barrio"
                    # Buscar los valores cod_barrio, distrito y cod_distrito en df_barrios
                    result = df_barrios.query(f"barrio_n == '{barrio_encontrado}'")
                    if not result.empty:
                        cod_distrito = result["cod_distrito"].values[0]
                        cod_barrio = result["cod_barrio"].values[0]
                        codigo_Postal=result["codigo_Postal"].values[0]
                        distrito_result = result["distrito"].values[0]
                        df.at[idx, "cod_barrio"] = cod_barrio
                        df.at[idx, "cod_distrito"] = cod_distrito
                        df.at[idx, "distrito"] = distrito_result
                        df.at[idx, "codigo_Postal"] = codigo_Postal

In [340]:
clasificar_distrito_y_barrio_por_columnas(df_unido, dfbarrios,'barrio')

In [341]:
df_unido.query("url=='https://www.pisosmadrid.com.es/propiedad/3890818-piso-en-venta-en-moratalaz-madrid'")

,title,url,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,consumoce_ano,letra_ce,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,antiguedad,num_plantas,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,exterior,interior,interior_y_exterior,piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,€_comunidad_mes,acceso_discapacitado,orientacion,precio,web,fecha_descarga,actualizacion,armarios,codigo_Postal,m2_utiles,referencia,tipo_suelo,certificado_energetico,tiene_jardin,descripcion_larga,num_visitas,fecha_publicacion,titularanuncio,num_pisos,puerta_seguridad,zonas_verdes,gastos_comunidad,tiene_piscina,tipo_fachada,agua_caliente,consumo_energia,nombre_oficina,letrace,portero,orientacion_este,orientacion_oeste,orientacion_sur,Lujo
9397,"Piso en venta en Moratalaz, Madrid",https://www.pisosmadrid.com.es/propiedad/3890818-piso-en-venta-en-moratalaz-madrid,NaN,NaN,NaN,69.00,Moratalaz,14.00,NaN,NaN,Moratalaz,NaN,NaN,NaN,NaN,Piso,NaN,NaN,NaN,NaN,3.00,1,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,nan,sin jardin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,198000,pisosmadrid,24/10/2024,NaN,NaN,NaN,NaN,3890818,NaN,NaN,0.00,"Piso 3 dormitorios, 1 baños, 0 garajes, Para reformar, en Madrid, Madrid Cuarto piso sin ascensor, distribuido en salón-comedor, cocina, tres dormitorios y un baño.",11.00,10/06/2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [342]:
#Guardo la union de los dos datasets
#df_unido.to_csv("20250111_Union_Sil_Ale_pisosmadrid.csv",index=False)
df_unido.to_csv("20250301_Union_todos.csv",index=False)
#df_union.to_feather("20241231_Union_Sil_Ale.ftr")

In [343]:
df_fichero= pd.read_excel('./data/arreglar_barrios_distritos.xlsx')

In [344]:
df_unido.query("cod_barrio.isnull()")[['url', 'distrito', 'cod_distrito', 'barrio', 'cod_barrio']]

,url,distrito,cod_distrito,barrio,cod_barrio
3896,https://www.pisos.com/comprar/piso-los_berrocales28052-4592114246_109700/,Vicálvaro,19.00,NaN,NaN
4062,https://www.pisos.com/comprar/piso-los_berrocales28031-4596004879_109700/,Vicálvaro,19.00,NaN,NaN
4063,https://www.pisos.com/comprar/piso-los_berrocales28031-4849665223_109700/,Vicálvaro,19.00,NaN,NaN
4064,https://www.pisos.com/comprar/piso-los_berrocales28031-4510709962_109700/,Vicálvaro,19.00,NaN,NaN
4065,https://www.pisos.com/comprar/piso-los_berrocales28031-4891530918_109700/,Vicálvaro,19.00,NaN,NaN
4194,https://www.pisos.com/comprar/piso-los_berrocales28031-3500496548_109700/,Vicálvaro,19.00,NaN,NaN
4195,https://www.pisos.com/comprar/piso-los_berrocales28031-4568166771_109700/,Vicálvaro,19.00,NaN,NaN
7614,https://www.pisos.com/comprar/piso-los_berrocales28031-4677433362_109700/,Vicálvaro,19.00,NaN,NaN
7615,https://www.pisos.com/comprar/piso-los_berrocales28031-4588865732_109700/,Vicálvaro,19.00,NaN,NaN
7616,https://www.pisos.com/comprar/piso-los_berrocales28031-4355464537_109700/,Vicálvaro,19.00,NaN,NaN


In [345]:
df_unido["barrio"]=df_unido["barrio"].str.strip()

In [346]:
df_unido.loc[df_unido["barrio"] == "Valdefuentes", ["cod_barrio", "cod_distrito", "codigoPostal"]] = [166, 16, 28050]


In [347]:
# Cambiar el valor del barrio
df_unido.loc[df_unido["barrio"] == "VillaverdeAlto,CascoHistóricodeVillaverde", "barrio"] = "Villaverde Alto"

# Asignar los nuevos valores a las columnas correspondientes
df_unido.loc[df_unido["barrio"] == "Villaverde Alto", ["cod_barrio", "cod_distrito", "codigoPostal"]] = [171, 17, 28021]


In [348]:
# Cambiar el valor del barrio
df_unido.loc[df_unido["barrio"] == "RECOLETOS", "barrio"] = "Recoletos"

# Asignar los nuevos valores a las columnas correspondientes
df_unido.loc[df_unido["barrio"] == "Recoletos", ["cod_barrio", "cod_distrito", "codigoPostal"]] = [41, 4, 28046]

In [349]:
# Cambiar el valor del barrio
df_unido.loc[df_unido["barrio"] == "SOL", "barrio"] = "Sol"

# Asignar los nuevos valores a las columnas correspondientes
df_unido.loc[df_unido["barrio"] == "Sol", ["cod_barrio", "cod_distrito", "codigoPostal"]] = [16, 1, 28005]

In [350]:
# Cambiar el valor del barrio
df_unido.loc[df_unido["barrio"] == "CASCO H.VALLECAS", "barrio"] = "Casco Histórico de Vallecas"

# Asignar los nuevos valores a las columnas correspondientes
df_unido.loc[df_unido["barrio"] == "Casco Histórico de Vallecas", ["cod_barrio", "cod_distrito", "codigoPostal"]] = [181, 18, 28031]

In [351]:
# Cambiar el valor del barrio
df_unido.loc[df_unido["barrio"] == "AEROPUERTO", "barrio"] = "Aeropuerto"

# Asignar los nuevos valores a las columnas correspondientes
df_unido.loc[df_unido["barrio"] == "Aeropuerto", ["cod_barrio", "cod_distrito", "codigoPostal"]] = [212, 21, 28042]

In [352]:
# Cambiar el valor del barrio
df_unido.loc[df_unido["barrio"] == "JUSTICIA", "barrio"] = "Justicia"

# Asignar los nuevos valores a las columnas correspondientes
df_unido.loc[df_unido["barrio"] == "Justicia", ["cod_barrio", "cod_distrito", "codigoPostal"]] = [14, 1, 28004]

In [353]:
# Cambiar el valor del barrio
df_unido.loc[df_unido["barrio"] == "Ángeles", "barrio"] = "Los Ángeles"

# Asignar los nuevos valores a las columnas correspondientes
df_unido.loc[df_unido["barrio"] == "Los Ángeles", ["cod_barrio", "cod_distrito", "codigoPostal"]] = [174, 17, 28021]

In [354]:
# Cambiar el valor del barrio
df_unido.loc[df_unido["barrio"] == "LOS CARMENES", "barrio"] = "Los Cármenes"

# Asignar los nuevos valores a las columnas correspondientes
df_unido.loc[df_unido["barrio"] == "Los Cármenes", ["cod_barrio", "cod_distrito", "codigoPostal"]] = [101, 10, 28047]

In [355]:
# Cambiar el valor del barrio
df_unido.loc[df_unido["barrio"] == "AMPOSTA", "barrio"] = "Amposta"

# Asignar los nuevos valores a las columnas correspondientes
df_unido.loc[df_unido["barrio"] == "Amposta", ["cod_barrio", "cod_distrito", "codigoPostal"]] = [203, 20, 28037]

In [356]:
# Cambiar el valor del barrio
df_unido.loc[df_unido["barrio"] == "VISTA ALEGRE", "barrio"] = "Vista Alegre"

# Asignar los nuevos valores a las columnas correspondientes
df_unido.loc[df_unido["barrio"] == "Vista Alegre", ["cod_barrio", "cod_distrito", "codigoPostal"]] = [114, 11, 28025]

In [357]:
#Guardo la union de los dos datasets
#df_unido.to_csv("20250111_Union_Sil_Ale_pisosmadrid.csv",index=False)
df_unido.to_csv("20250301_Union_todos.csv",index=False)
#df_union.to_feather("20241231_Union_Sil_Ale.ftr")

In [358]:
df_unido['barrio'] = df_unido['barrio'].str.lower().str.title()

In [359]:
df_unido.query("cod_barrio==193")[["url"]]

,url
5547,https://www.pisos.com/comprar/piso-valderrivas28032-42585938653_994867/
7589,https://www.pisos.com/comprar/piso-valderrivas28032-42587290865_105700/
7590,https://www.pisos.com/comprar/piso-valderrivas28032-42581466648_100500/


In [360]:
df_unido["barrio"].value_counts(dropna=False)

barrio
Universidad                     449
Goya                            447
Sol                             354
Castellana                      337
Recoletos                       335
Palacio                         279
Justicia                        267
Embajadores                     256
El Pardo                        234
Lista                           211
Ibiza                           205
Cortes                          185
Almagro                         171
Hispanoamérica                  144
Aluche                          143
El Viso                         139
Pueblo Nuevo                    135
Prosperidad                     132
Cuatro Caminos                  123
Nueva España                    122
Trafalgar                       122
San Diego                       119
Numancia                        117
Argüelles                       116
Simancas                        111
Vista Alegre                    104
Fuente Del Berro                103
Villaverde Alto      

In [361]:
reemplazos = {'Villaverde Alto C.H.': 'Villaverde Alto', 'Casco H.Vicalvaro': 'Casco Histórico De Vicálvaro', 'Arguelles': 'Argüelles','Los Carmenes':'Los Cármenes'
,'Puerta Del Angel':'Puerta Del Ángel','Casco H.Vallecas':'Casco Histórico De Vallecas','Apóstol Santiago':'Apostol Santiago','Corralejos':'Corralejo'}
df_unido['barrio'] = df_unido['barrio'].replace(reemplazos)

In [362]:
df_unido['distrito'] = df_unido['distrito'].str.lower().str.title().str.strip()

In [363]:
# Definir el diccionario de reemplazos
reemplazos = {
    'Chamberi': 'Chamberí',
    'Chamartin': 'Chamartín',
    'Tetuan': 'Tetuán',
    'Vicalvaro': 'Vicálvaro',
    'Fuencarral El Pardo':'Fuencarral-El Pardo'

}

# Reemplazar múltiples valores en la columna 'distrito'
df_unido['distrito'] = df_unido['distrito'].replace(reemplazos).str.strip()


In [364]:
df_unido["distrito"].value_counts(dropna=False)

distrito
Centro                 1809
Salamanca              1533
Chamartín               640
Chamberí                612
Carabanchel             541
Puente De Vallecas      530
Tetuán                  467
Fuencarral-El Pardo     432
Retiro                  428
Latina                  416
Ciudad Lineal           409
Moncloa-Aravaca         380
San Blas-Canillejas     351
Arganzuela              298
Hortaleza               291
Villaverde              290
Villa De Vallecas       233
Usera                   196
Vicálvaro               137
Moratalaz               105
Barajas                 100
NaN                       7
Name: count, dtype: int64

In [365]:
df_unido.cod_distrito.value_counts(dropna=False)

cod_distrito
 1       1820
 4       1533
5.00      640
7.00      612
11.00     538
13.00     531
6.00      467
8.00      432
3.00      427
10.00     415
15.00     407
9.00      380
20.00     350
2.00      298
 16       291
17.00     290
18.00     232
12.00     194
19.00     137
14.00     104
21.00     100
 NaN        7
Name: count, dtype: int64

In [366]:
df_unido.cod_barrio.value_counts(dropna=False)

cod_barrio
15.00     449
42.00     447
 16       354
46.00     337
 41       335
11.00     279
 14       267
12.00     256
81.00     234
45.00     211
34.00     205
13.00     185
74.00     171
54.00     144
104.00    143
51.00     138
152.00    134
52.00     132
62.00     123
55.00     122
73.00     122
132.00    119
92.00     117
136.00    117
201.00    111
114.00    104
43.00     103
171.00     99
35.00      97
44.00      95
 166       93
183.00     92
131.00     89
76.00      86
61.00      86
75.00      86
63.00      86
116.00     85
115.00     84
207.00     80
56.00      80
71.00      79
181.00     79
112.00     78
134.00     73
93.00      73
117.00     73
163.00     73
25.00      71
102.00     70
107.00     69
135.00     69
 NaN       68
175.00     67
83.00      67
113.00     66
97.00      65
151.00     63
72.00      63
22.00      62
66.00      59
133.00     59
182.00     58
164.00     56
103.00     56
172.00     55
153.00     54
174.00     52
194.00     52
65.00      51
31.00    

In [367]:
df_unido["codigo_Postal"].value_counts(dropna=False)

codigo_Postal
28001.00    781
28013.00    631
28015.00    580
28006.00    548
28037.00    348
28002.00    300
28010.00    291
28021.00    287
28014.00    280
28025.00    273
28004.00    267
28016.00    266
28012.00    255
28027.00    240
28047.00    236
28009.00    236
28048.00    233
28031.00    225
28020.00    209
28053.00    205
28018.00    200
28045.00    200
28028.00    198
28039.00    192
28019.00    186
28041.00    182
28003.00    171
28029.00    160
28017.00    135
28032.00    133
28023.00    123
 NaN        122
28043.00    122
28035.00    118
28038.00    117
28008.00    116
28005.00    100
28042.00     99
28055.00     98
28030.00     98
28033.00     94
28007.00     93
28011.00     91
28026.00     90
28046.00     80
28040.00     71
28044.00     69
28024.00     35
28034.00     20
28049.00      1
Name: count, dtype: int64

In [368]:
# Llamar a la función
df_unido = completar_codigo_postal(df_unido, dfbarrios)

In [369]:

df_unido['codigo_Postal'] = pd.to_numeric(df_unido['codigo_Postal'], errors='coerce').dropna().astype('Int64')

In [370]:
# Supongamos que ya tienes un DataFrame llamado df
columnas_nulas = df_unido.columns[df_unido.isnull().all()]

print("Columnas con todos los valores nulos:")
print(columnas_nulas)

Columnas con todos los valores nulos:
Index(['Lujo'], dtype='object')


In [371]:
df_unido["disponibilidad"].value_counts(dropna=False)

disponibilidad
NaN      9768
Libre     437
Name: count, dtype: int64

In [372]:
#cols_eliminar=['usufructo', 'derecho_superficie', 'buhardilla', 'hidromasaje' , 'orientacion_norte,_sur','orientacion_sur,_este', 
#'Lujo','propiedad_completa','disponibilidad','instalacion_tv_centralizada','instalacion_tv_individual' ]
cols_eliminar=[ 'Lujo','propiedad_completa','disponibilidad' ]
df_unido=df_unido.drop(columns=cols_eliminar)

In [373]:
#df_unido.loc[df_unido['sistema_alarma'] == 1, 'alarma'] = 1

In [374]:
df_unido['portero'] = df_unido['portero'].astype(object)
# Asignar valores de 'porteria' a 'portero' cuando 'porteria' no sea nula
df_unido.loc[df_unido['porteria'].notnull(), 'portero'] = df_unido['porteria']

In [375]:
df_unido.query("title.isnull()")[["web"]]

,web


In [376]:
#elimino "codigoPostal" por que los valores estan en codigo_Postal
celim=["codigoPostal","porteria","sup_comercial"]
df_unido=df_unido.drop(columns=celim)

In [377]:
df_unido.columns.tolist()

['title',
 'url',
 'precio_anterior',
 'descuento',
 'EUR/m2',
 'm2_constr',
 'distrito',
 'cod_distrito',
 'barrio',
 'cod_barrio',
 'zona',
 'calle',
 'consumoce_ano',
 'letra_ce',
 'emisiones_co2',
 'tipologia',
 'estado',
 'amueblado',
 'planta',
 'dormitorios',
 'banos',
 'balcon',
 'terraza',
 'garaje',
 'trastero',
 'ascensor',
 'calefaccion',
 'antiguedad',
 'num_plantas',
 'aire_acondicionado',
 'jardin',
 'inmueble_ingresos',
 'alquiler_opcion_a_compra',
 'nuda_propiedad',
 'tipo_inmueble',
 'tiene_armario',
 'cancha_tenis',
 'carpinteria_exterior_doble_vidrio/pvc',
 'carpinteria_exterior_doble_vidrio/madera',
 'carpinteria_exterior_doble_vidrio/metal',
 'carpinteria_exterior_triple_vidrio/madera',
 'carpinteria_exterior_triple_vidrio/metal',
 'carpinteria_exterior_vidrio/pvc',
 'carpinteria_exterior_vidrio/madera',
 'carpinteria_exterior_vidrio/metal',
 'chimenea',
 'cocina',
 'fibra_optica',
 'exterior',
 'interior',
 'interior_y_exterior',
 'piscina',
 'porton_electrico',


In [378]:
df_unido.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10205 entries, 0 to 10204
Data columns (total 89 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   title                                      10205 non-null  object 
 1   url                                        10205 non-null  object 
 2   precio_anterior                            55 non-null     float64
 3   descuento                                  55 non-null     object 
 4   EUR/m2                                     7855 non-null   float64
 5   m2_constr                                  10198 non-null  float64
 6   distrito                                   10198 non-null  object 
 7   cod_distrito                               10198 non-null  object 
 8   barrio                                     10137 non-null  object 
 9   cod_barrio                                 10137 non-null  object 
 10  zona                  

In [379]:
df_unido.to_csv("20250301_Union_todos.csv",sep=";",index=False)

In [380]:
df_fichero= pd.read_excel('./data/arreglar_barrios_distritos.xlsx')

#df_fichero= pd.read_excel('./data/arreglar_barrios_distritos.xlsx')
# Asegurar que las columnas necesarias existen en ambos DataFrames
columnas = ['url', 'distrito', 'cod_distrito', 'barrio', 'cod_barrio']
df_fichero = df_fichero[columnas].dropna(subset=['url'])  # Eliminar filas sin URL
n=0
# Recorrer el fichero y actualizar df_unido
for _, row in df_fichero.iterrows():
    url = row['url']
    print(url)
    # Verificar si la URL está en df_unido
    if url in df_unido['url'].values:
        n=n+1
        # Encontrar el índice de la fila correspondiente en df_unido
        index = df_unido[df_unido['url'] == url].index[0]
       # print(index)
        # Actualizar solo los valores no nulos en el fichero
        for col in columnas[1:]:  # Excluir 'url'
            if pd.notna(row[col]):  # Si en el fichero hay un valor válido
                df_unido.at[index, col] = row[col]

print(n)